In [127]:
import requests
import pandas as pd
from io import StringIO
from typing import Any
import time


In [128]:
def generateRange(init=2020, end=2022):
    res = []
    for num in range(init, end + 1):
        res.append(num)
    return res


def makeReq(payload: dict[str, Any]):
    headers = {
        "Authorization": "8365b04a-652c-490e-ac85-1b693d606266 ",
        "Content-Type": "application/json",
        "Accept": "text/csv",
    }

    r = requests.post(
        "https://www.presupuestoabierto.gob.ar/api/v1/credito",
        json=payload,
        headers=headers,
    )

    if r.status_code != 200:
        raise Exception(f"Status code isn't 200, res text:\n{r.text}")

    return r.text


def programa(rango: list[int]) -> str:
    payload = {
        "ejercicios": rango,
        "columns": [
            "ejercicio_presupuestario",
            "impacto_presupuestario_anio",
            "entidad_id",
            "entidad_desc",
            "programa_id",
            "programa_desc",
            "credito_presupuestado",
            "credito_vigente",
            "credito_devengado",
            "credito_comprometido" "credito_pagado",
        ],
    }
    text = makeReq(payload)
    return text


def finalidadFuncion(rango: list[int]) -> str:
    payload = {
        "ejercicios": rango,
        "columns": [
            "ejercicio_presupuestario",
            "finalidad_id",
            "finalidad_desc",
            "funcion_id",
            "funcion_desc",
            "credito_presupuestado",
            "credito_vigente",
            "credito_devengado",
            "credito_comprometido",
            "credito_pagado",
        ],
    }
    text = makeReq(payload)
    return text


In [129]:
dfs = []
year = 2012
for num in range(0, 11):
    print(year)
    csv = finalidadFuncion([year])
    dfs.append(pd.read_csv(StringIO(csv)))
    time.sleep(2)
    year+=1



2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022


In [130]:
presupuesto = pd.concat(dfs, axis=0)
presupuesto.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 319 entries, 0 to 28
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ejercicio_presupuestario  319 non-null    int64  
 1   finalidad_id              319 non-null    int64  
 2   finalidad_desc            319 non-null    object 
 3   funcion_id                319 non-null    int64  
 4   funcion_desc              319 non-null    object 
 5   credito_presupuestado     319 non-null    float64
 6   credito_vigente           319 non-null    float64
 7   credito_devengado         319 non-null    float64
 8   credito_comprometido      319 non-null    float64
 9   credito_pagado            319 non-null    float64
dtypes: float64(5), int64(3), object(2)
memory usage: 27.4+ KB


In [131]:
presupuesto.groupby("ejercicio_presupuestario").sum()

,finalidad_id,funcion_id,credito_presupuestado,credito_vigente,credito_devengado,credito_comprometido,credito_pagado
ejercicio_presupuestario,,,,,,,
2012,77,119,5.051300e+05,5.885867e+05,5.618330e+05,5.627638e+05,5.318703e+05
2013,77,119,6.287127e+05,7.801498e+05,7.509349e+05,7.519887e+05,7.019810e+05
2014,77,119,8.595817e+05,1.164759e+06,1.134364e+06,1.135193e+06,1.081679e+06
2015,77,119,1.251675e+06,1.499476e+06,1.431662e+06,1.433986e+06,1.358030e+06
2016,77,119,1.570128e+06,2.215683e+06,2.131113e+06,2.133472e+06,2.054866e+06
2017,77,119,2.363659e+06,2.633515e+06,2.564005e+06,2.566868e+06,2.482121e+06
2018,77,119,2.878796e+06,3.379235e+06,3.298979e+06,3.300665e+06,3.193254e+06
2019,77,119,4.172312e+06,4.928184e+06,4.762089e+06,4.765882e+06,4.587236e+06
2020,77,119,4.860072e+06,7.762601e+06,7.001429e+06,7.004202e+06,6.777291e+06


In [132]:
presupuesto.to_csv("./data/presupuesto/2012-2022-finalidad-funcion.csv", index=False)